In [1]:
from music21 import *
import numpy as np
from hmmlearn.hmm import MultinomialHMM
import os
import pickle

In [2]:
# Analyze Bach music dataset
all_bach_paths = corpus.getComposer('bach')
print("Total number of Bach pieces to process from music21:", len(all_bach_paths))

parsed_bach = []
keys = []
songs_by_key = {}
for i, p_bach in enumerate(all_bach_paths):
    p = corpus.parse(p_bach)
    if len(p.parts) != 4:
        continue
    parsed_bach.append(p)
    if p.analyze('key') not in keys:
        keys.append(p.analyze('key'))
        songs_by_key[p.analyze('key')] = [p]
    else:
        songs_by_key[p.analyze('key')].append(p)
        
print("Updated number of Bach pieces to process from music21:", len(parsed_bach))
for key in keys:
    print("Number of songs in", key, "key:", len(songs_by_key[key]))

Total number of Bach pieces to process from music21: 433
Updated number of Bach pieces to process from music21: 368
Number of songs in g minor key: 43
Number of songs in d minor key: 29
Number of songs in c minor key: 10
Number of songs in b minor key: 27
Number of songs in A major key: 32
Number of songs in D major key: 25
Number of songs in a minor key: 47
Number of songs in G major key: 45
Number of songs in C major key: 14
Number of songs in e minor key: 19
Number of songs in F major key: 25
Number of songs in B- major key: 22
Number of songs in E major key: 9
Number of songs in E- major key: 9
Number of songs in f# minor key: 7
Number of songs in f minor key: 2
Number of songs in A- major key: 1
Number of songs in b- minor key: 2


In [2]:
if os.path.exists('transposed_music.pkl'):
    with open('transposed_music.pkl', 'rb') as file:
        transposed_bach_major, transposed_bach_minor = pickle.load(file)
else:
    # Transpose all songs into C
    transposed_bach_major = []
    transposed_bach_minor = []
    for p in parsed_bach:
        k = p.analyze('key')
        i = interval.Interval(k.tonic, pitch.Pitch('C'))
        if 'major' in k.name:
            transposed_bach_major.append(p.transpose(i))
        elif 'minor' in k.name:
            transposed_bach_minor.append(p.transpose(i))
    # Save as pickle so we don't need to again
    with open('transposed_music.pkl', 'wb') as file:
        pickle.dump([transposed_bach_major, transposed_bach_minor], file)

In [3]:
class Song:
    def __init__(self):
        """Initialize Song class.
        
        Attributes:
            score (Score): Song from music21 Score class
            key (str): Key of the song
            parts (ndarray(p,n)): Array of parts(p) and notes(n)
            n_parts (int): Number of parts in the song
            stream (Stream): music21 Stream object generated from parts
        """
        return
        
    def parse(self, score):
        """Parse the MusicXML score into a trainable format"""
        parts = []
        discrete = [[] for _ in range(len(score.parts))]
        
        # Generate discretized notelist for each part
        for i, p in enumerate(score.parts):
            for n in p.recurse().notesAndRests:
                if n.isRest:
                    discrete[i] = discrete[i] + ([0] * int(n.quarterLength*12))
                else:
                    discrete[i] = discrete[i] + ([n.pitch] * int(n.quarterLength*12))
        
        # Generate states from the music
        for i, p in enumerate(score.parts):
            t = 0         # Time for dependence on other parts
            notes = []    # States (pitch[0,t], ..., pitch[n_parts,t], dur)
            for n in p.recurse().notesAndRests:
                pitches = [part[t] for part in discrete]
                notes.append(pitches + [n.quarterLength])
                t += int(n.quarterLength*12)
                
            # Add notes list for each part to part list
            parts.append(notes)  
            
            self.score = score
            self.key = score.analyze('key')
            self.parts = np.array(parts, dtype=object)
            self.n_parts = len(parts)  
        
    def gen_stream(self, parts=None):
        """Generate a song from the parts and notes parsed
        
        Parameters:
            parts (list): list of part indices to generate
        """
        # If None, generate all parts
        if parts is None:
            parts = np.arange(self.n_parts)
        parts = np.array(parts, dtype=int)
        
        # Generate the song for the parts indicated
        s = stream.Stream()
        for i, notes in enumerate(self.parts[parts]):
            p = stream.Part(id=i)
            for n in notes: 
                if n[i] == 0:
                    p.append(note.Rest(quarterLength=n[self.n_parts]))
                else:
                    p.append(note.Note(n[i],quarterLength=n[self.n_parts]))
            s.insert(0,p)
        self.stream = s
        
    def play(self):
        """Play the song generated by gen_stream"""
        self.stream.show('midi')  
        
    def save(self, filename):
        """Save the generated song as a midi file"""
        self.stream.write('midi', filename)

In [41]:
class MusicHMM:
    def __init__(self, n_components):
        """Initialize MusicHMM class
        
        Attributes:
            songs (list): List of Song class objects to train on
            n_components (int): number of components for the HMM
            HMMs (list(MultinomialHMM)): HMM objects for each part
            states (ndarray): Array of possible states
            state_ind (func): Map from note state to index
            obs (list): Observed sequences of states by index
            obs_len (list): length of observation sequence for each song
        """
        self.n_components = n_components
        
        
    def fit(self, songs, parts=[0,1,2,3]):
        """Train the HMM on the list of songs for part indices given
        
        Parameters:
            songs (list): List of parsed Song objects to train on
            parts (list): List of part indices from songs to use in training
        """
        self.songs = songs
        
        # Generate state space data and observations by index
        self.init_states(parts)
        self.init_obs_matrices(parts)
        
        # Train a HMM for each part
        n_parts = len(parts)
        HMMs = []
        for i in range(n_parts):
            obs = np.array(self.obs[i]).reshape(-1, 1)   # Reshape observations
            hmm = MultinomialHMM(n_components=self.n_components)
            hmm.fit(obs, lengths=self.obs_len[i])
            HMMs.append(hmm)
            
        self.HMMs = HMMs
        
    def init_states(self, parts):
        """Create and save a dictionary of unique note states"""
        # Initialize
        n_parts = len(parts)
        states = [[] for _ in range(n_parts)]
        
        for song in self.songs:
            for i, p in enumerate(song.parts[parts]):
                for n in p:
                    if n not in states[i]:
                        states[i].append(n)
                    
        self.states = states
        self.n_parts = n_parts
        
        
    def init_obs_matrices(self, parts):
        """Create the matrices of note indices observed"""
        # Initialize
        obs = [[] for _ in range(self.n_parts)]
        obs_len = [[] for _ in range(self.n_parts)]
        
        for i in range(self.n_parts):
            for song in self.songs:
                p = song.parts[parts][i]
                seq = [self.states[i].index(n) for n in p]
                obs[i] += seq
                obs_len[i].append(len(seq))
                
        self.obs = obs
        self.obs_len = obs_len 
                
               
    def gen_song(self, measures=12):
        """Sample a new song from the HMM
        
        Parameters:
            num_notes (int): Number of notes to sample from each part
            parts (list): List of part indices to generate in song
        """
        s = stream.Score()
        part_lengths = []
        for i in range(self.n_parts):
            hmm = self.HMMs[i]
            ind = hmm.sample(measures*16)[0].ravel()
            states = np.array(self.states[i])[ind]
            
            k = 0
            for m in range(measures):
                T = 0
                while T != 4:
                    d = states[k][-1]
                    if T + d > 4:
                        d = 4 - T
                    states[k][-1] = d
                    T += d
                    k += 1
            states = states[:k]
            
            p = stream.Part(id=i)
            for n in states: 
                if n[i] == 0:
                    p.append(note.Rest(quarterLength=n[-1]))
                else:
                    p.append(note.Note(n[i],quarterLength=n[-1]))
            s.insert(0,p)
            
        new_song = Song()
        new_song.parse(s)
        new_song.gen_stream()
        
        return new_song
    
    
    def dep_gen_song(self, measures=12, measure_len=4):
        """Sample a new song from the HMM with dependence between parts
        
        Parameters:
            num_notes (int): Number of notes to sample from each part
            parts (list): List of part indices to generate in song
        """
        def measureize(obs, measures, measure_len=4):
            """Return measure-ized sequence of notes"""
            k = 0       # Index for observation states
            max_time = 12 * measure_len
            for m in range(measures):
                T = 0
                while T != max_time:
                    d = int(obs[k][-1] * 12)
                    if T + d > max_time:
                        d = max_time - T
                        obs[k] = [obs[k][l] for l in range(self.n_parts)] + [float(d) / 12]
                    T += d
                    k += 1
                    
            return obs[:k]
            
        
        # Generate states from trained models
        hidden_seq = []      # List of hidden states sampled
        for i in range(self.n_parts):
            hmm = self.HMMs[i]
            Z, X = hmm.sample(measures*measure_len*12)
            if i == (-1 % self.n_parts):
                bass_seq =  Z.ravel()
            else:
                hidden_seq.append(X)
            
        # Measure-ize bass
        bass = measureize(np.array(self.states[-1])[bass_seq], measures, measure_len)
            
        # Discretize bass states
        bass_disc = []
        for n in bass:
            bass_disc += [n[-2]] * int(n[-1] * 12)
            
        # Generate dependent states while measurizing
        note_seq = [[] for _ in range(self.n_parts-1)] + [bass.tolist()]
        conditional_notes = {}
        K = [0 for _ in range(self.n_parts-1)]
        max_time = 12 * measure_len
        for m in range(measures):
            for i in range(self.n_parts-1):
                part_states = self.states[i]
                x_states = hidden_seq[i]          # Generated hidden states
                hmm = self.HMMs[i]
                
                T = 0
                while T < max_time:
                    cond = bass_disc[T]           # Find bass note condition

                    # Find note state indices that satisfy the bass note condition
                    if (i,cond) not in conditional_notes.keys():
                        cond_ind = [j for j in range(len(part_states)) 
                                    if (part_states[j][-2]==cond)]
                        conditional_notes[(i,cond)] = cond_ind
                    else:
                        cond_ind = conditional_notes[(i,cond)]
                    cond_ind = np.array(cond_ind)
                    
                    # What if there are no notes conditionally?
                    if len(cond_ind) > 0:
                        # Get conditional probabilities
                        prob = np.array(hmm.emissionprob_[x_states[K[i]], cond_ind])
                        cond_prob = prob / np.sum(prob)
                        
                        # Generate a note conditioned on the discretized bass
                        note_ind = cond_ind[np.random.choice(np.arange(len(prob)), 
                                                             size=1, p=cond_prob)]
                        n = part_states[note_ind[0]]
                    else:
                        n = [0 for _ in range(self.n_parts-1)] + [cond, 1.0]
                    
                    # Adjust timing for measurization
                    d = int(n[-1] * 12)
                    if T + d > max_time:
                        d = max_time - T
                        n = [n[l] for l in range(self.n_parts)] + [float(d) / 12]
                    T += d
                    
                    K[i] += 1
                    note_seq[i].append(n)
                    
        # Cheat - Create a final C chord to end the song
        chord = [pitch.Pitch('C4'), pitch.Pitch('E3'), 
                 pitch.Pitch('G2'), pitch.Pitch('C2'), 4.0]
        for p in note_seq:
            p.append(chord)
        
        
                
        # Create the Song object
        s = stream.Score()
        for i in range(self.n_parts):
            p = stream.Part(id=i)
            for n in note_seq[i]:
                if n[i] == 0:
                    p.append(note.Rest(quarterLength=n[-1]))
                else:
                    p.append(note.Note(n[i],quarterLength=n[-1]))
            s.insert(0,p)
            
        new_song = Song()
        new_song.parse(s)
        new_song.gen_stream()
        
        return new_song

In [34]:
# Test on one song
song = Song()
song.parse(transposed_bach_major[0])
song.gen_stream()
song.play()

In [46]:
# Test on one song
song1 = Song()
song1.parse(transposed_bach_major[1])
song1.gen_stream()
song1.play()

In [50]:
# Train on one song
mhmm = MusicHMM(20)
mhmm.fit([song, song1], parts=[0,1,2,3])
new_song = mhmm.dep_gen_song(1, 48)
new_song.play()

Fitting a model with 1979 free scalar parameters with only 96 data points will result in a degenerate solution.
Fitting a model with 2419 free scalar parameters with only 121 data points will result in a degenerate solution.
Fitting a model with 2519 free scalar parameters with only 119 data points will result in a degenerate solution.
Fitting a model with 2539 free scalar parameters with only 129 data points will result in a degenerate solution.


In [36]:
# Save file, load from there if it exists, because this takes a hot second to run
filename = 'processed_music.pkl'
if os.path.exists(filename):
    with open(filename, 'rb') as file:
        maj_songlist, min_songlist = pickle.load(file)
else:
    # Create song classes for all transposed Bach music
    maj_songlist = []
    min_songlist = []

    for p in transposed_bach_major:
        song = Song()
        song.parse(p)
        maj_songlist.append(song)

    for p in transposed_bach_minor:
        song = Song()
        song.parse(p)
        min_songlist.append(song)
    
    with open(filename, 'wb') as file:
        pickle.dump([maj_songlist, min_songlist], file)

print("Number of major songs:", len(maj_songlist))
print("Number of minor songs:", len(min_songlist))

Number of major songs: 182
Number of minor songs: 186


## Train on Major songs

In [42]:
# Train on songlist of transposed music
filename = 'maj_mhmm.pkl'
if os.path.exists(filename):
    with open(filename, 'rb') as file:
        maj_mhmm = pickle.load(file)
else:
    maj_mhmm = MusicHMM(100)
    maj_mhmm.fit(maj_songlist)
    # Save as pickle so we don't need to again
    with open(filename, 'wb') as file:
        pickle.dump(maj_mhmm, file)

In [45]:
new_song = maj_mhmm.dep_gen_song(2, 24)
new_song.play()
new_song.save("generated_music/dep_maj_song1.midi")

In [44]:
new_song = maj_mhmm.dep_gen_song(12, 4)
new_song.play()
new_song.save("generated_music/dep_maj_song2.midi")

## Train on Minor songs

In [13]:
# Train on songlist of transposed music
min_mhmm = MusicHMM(100)
min_mhmm.fit(maj_songlist)

Fitting a model with 281399 free scalar parameters with only 9436 data points will result in a degenerate solution.
Fitting a model with 343299 free scalar parameters with only 10957 data points will result in a degenerate solution.
Fitting a model with 345599 free scalar parameters with only 11207 data points will result in a degenerate solution.
Fitting a model with 334799 free scalar parameters with only 11595 data points will result in a degenerate solution.


In [14]:
new_song = min_mhmm.gen_song(num_notes=40)
new_song.play()
new_song.save("generated_music/min_song1.midi")

In [15]:
new_song = min_mhmm.gen_song(num_notes=40)
new_song.play()
new_song.save("generated_music/min_song2.midi")